In [208]:
import glob
import pandas as pd
import os
from datetime import datetime

# Описание задачи

В нашей сети около 100 аптек. Нам нужно единоразово загрузить файл в 1С, но для этого надо сделать **сравнение реализации по аптекам** и **выгрузки из СБИСа**.

1. Выгрузка из СБИС находится в папке `Входящие`. Загрузите все csv файлы в один датафрейм с определенным набором столбцов.

2. Сравнение реализации по аптекам: Файлы с выгрузкой из аптек находятся в папке `Аптеки/csv/correct/`.

Загружайте на обработку по одному файлу. Содержимое каждого файла загружайте в датафрейм Pandas.

Столбцы датафрейма:
```
"Номер счет-фактуры"
"Сумма счет-фактуры"
"Дата счет-фактуры"
"Сравнение дат"

```

* Если "Поставщик" - ЕАПТЕКА, то к "Номер накладной" нужно добавить /15.

* Нужно найти все записи в выгрузке из СБИСа по данному номеру накладной: из найденных строк нужно оставить только те, которые имеют один из типов документа: ["СчФктр", "УпдДоп", "УпдСчфДоп", "ЭДОНакл"]. Если найдено - нужно сохранить значения Номер, Сумма и Дата

* Дату нужно представить в формате 25.05.2021

* В столбец Сравнение дат помещаем "Не совпадает!", если найденная дата и дата накладной отличаются. Иначе - пустая строка. 

Итоговый файл выгрузки по аптеке должен содержать такие столбцы в заданном порядке:

```
'№ п/п', 'Штрих-код партии', 'Наименование товара', 'Поставщик',
'Дата приходного документа', 'Номер приходного документа',
'Дата накладной', 'Номер накладной', 'Номер счет-фактуры',
'Сумма счет-фактуры', 'Кол-во',
'Сумма в закупочных ценах без НДС', 'Ставка НДС поставщика',
'Сумма НДС', 'Сумма в закупочных ценах с НДС', 'Дата счет-фактуры', 'Сравнение дат'

```

Файл сохраните по пути `Результат/{полная сегодняшняя дата}/{имя исходного файла без расширения} - результат.xlsx`. Если таких папок не существует - создайте их

## Выгрузка из СБИС

Для начала зададим столбцы для датафрейма, в который будет происходить выгрузка

In [123]:
cols = ["Дата",
        "Номер",
        "Сумма",
        "Статус",
        "Примечание",
        "Комментарий",
        "Контрагент",
        "ИНН/КПП",
        "Организация",
        "ИНН/КПП 1",
        "Тип документа",
        "Имя файла",
        "Дата 1",
        "Номер 1",
        "Сумма 1",
        "Сумма НДС",
        "Ответственный",
        "Подразделение",
        "Код",
        "Дата 2",
        "Время",
        "Тип пакета",
        "Идентификатор пакета",
        "Запущено в обработку",
        "Получено контрагентом",
        "Завершено",
        "Увеличение суммы",
        "НДC",
        "Уменьшение суммы",
        "НДС 1"]
cols = [name.replace(' ', '_') for name in cols]

In [124]:
sbis_df = pd.DataFrame(columns=cols)
sbis_df

,Дата,Номер,Сумма,Статус,Примечание,Комментарий,Контрагент,ИНН/КПП,Организация,ИНН/КПП_1,...,Время,Тип_пакета,Идентификатор_пакета,Запущено_в_обработку,Получено_контрагентом,Завершено,Увеличение_суммы,НДC,Уменьшение_суммы,НДС_1


Данные для выгрузки необходимо взять из папки `Входящие`, причем выгружать необходимо файлы с форматом .csv

In [125]:
total_rows = 0
for file in glob.glob('Входящие/*.csv'):
    df = pd.read_csv(file, encoding = '1251', sep = ';')
    total_rows += len(df)
    df.columns = cols
    sbis_df = pd.concat([sbis_df, df], axis=0)
print("Строк в найденных для выгрузки файлов:", total_rows)
print("Строк в итоговом датафрейме sbis_df:", len(sbis_df))

Строк в найденных для выгрузки файлов: 295737
Строк в итоговом датафрейме sbis_df: 295737


In [126]:
sbis_df.head()

,Дата,Номер,Сумма,Статус,Примечание,Комментарий,Контрагент,ИНН/КПП,Организация,ИНН/КПП_1,...,Время,Тип_пакета,Идентификатор_пакета,Запущено_в_обработку,Получено_контрагентом,Завершено,Увеличение_суммы,НДC,Уменьшение_суммы,НДС_1
0,30.09.21,БРН00709545,"4 056,46",Выполнение завершено успешно,NaN,"БРН00709545 на сумму 4 056,46р.","Пульс Брянск, ООО",3255510243 / 325701001,ООО Рога и Копыта,4025419873 / 402501001,...,23:40:56,ДокОтгрВх,76210df0-7c48-4e6d-ad2b-6cbe2f3b7722,30.09.21 23:40,01.10.21 08:39,01.10.21 14:59,"0,00","0,00","0,00","0,00"
1,30.09.21,БРН00709545,"4 056,46",Выполнение завершено успешно,NaN,"БРН00709545 на сумму 4 056,46р.","Пульс Брянск, ООО",3255510243 / 325701001,ООО Рога и Копыта,4025419873 / 402501001,...,23:40:56,ДокОтгрВх,76210df0-7c48-4e6d-ad2b-6cbe2f3b7722,NaN,NaN,NaN,"0,00","0,00","0,00","0,00"
2,30.09.21,БРН00709545,"0,00",Выполнение завершено успешно,NaN,"БРН00709545 на сумму 4 056,46р.","Пульс Брянск, ООО",3255510243 / 325701001,ООО Рога и Копыта,4025419873 / 402501001,...,23:40:56,ДокОтгрВх,76210df0-7c48-4e6d-ad2b-6cbe2f3b7722,NaN,NaN,NaN,"0,00","0,00","0,00","0,00"
3,30.09.21,БРН00709520,"54 705,65",Выполнение завершено успешно,NaN,"БРН00709520 на сумму 54 705,65р.","Пульс Брянск, ООО",3255510243 / 325701001,ООО Рога и Копыта,4025419873 / 402501001,...,23:34:51,ДокОтгрВх,1cfdcbd4-a43b-4543-90b6-59d0e6095b94,30.09.21 23:34,01.10.21 08:39,01.10.21 14:59,"0,00","0,00","0,00","0,00"
4,30.09.21,БРН00709520,"54 705,65",Выполнение завершено успешно,NaN,"БРН00709520 на сумму 54 705,65р.","Пульс Брянск, ООО",3255510243 / 325701001,ООО Рога и Копыта,4025419873 / 402501001,...,23:34:51,ДокОтгрВх,1cfdcbd4-a43b-4543-90b6-59d0e6095b94,NaN,NaN,NaN,"0,00","0,00","0,00","0,00"


## Сравнение реализации по аптекам

In [166]:
drugstores_df = pd.DataFrame()
total_rows = 0
for file in glob.glob('Аптеки/csv/correct/*.csv'):
    df = pd.read_csv('Аптеки/csv/correct/А123.csv', encoding = '1251', sep =';')
    total_rows += len(df)
    df.loc[df["Поставщик"].str.contains("ЕАПТЕКА", na=False), "Номер накладной"] = df["Номер накладной"].astype(str) + '/15'
    drugstores_df = pd.concat([drugstores_df, df], axis = 0)
print("Строк в найденных для выгрузки файлов:", total_rows)
print("Строк в итоговом датафрейме drugstores_df:", len(drugstores_df))


Строк в найденных для выгрузки файлов: 9638
Строк в итоговом датафрейме drugstores_df: 9638


In [167]:
drugstores_df.columns

Index(['№ п/п', 'Штрих-код партии', 'Наименование товара', 'Поставщик',
       'Дата приходного документа', 'Номер приходного документа',
       'Дата накладной', 'Номер накладной', 'Кол-во',
       'Сумма в закупочных ценах без НДС', 'Ставка НДС поставщика',
       'Сумма НДС', 'Сумма в закупочных ценах с НДС'],
      dtype='object')

In [168]:
drugstores_df.head()

,№ п/п,Штрих-код партии,Наименование товара,Поставщик,Дата приходного документа,Номер приходного документа,Дата накладной,Номер накладной,Кол-во,Сумма в закупочных ценах без НДС,Ставка НДС поставщика,Сумма НДС,Сумма в закупочных ценах с НДС
0,1,201300039533,СФМ ПЕРЧАТКИ СМОТР. ГОЛУБЫЕ Н/СТЕР. НИТРИЛ Р.L...,Катрен г.Химки,19.04.2021,13001265.0,18.04.2021,3541290-30,0.020000,36.48,10%,3.65,40.13
1,2,201300052321,ХАРТМАНН КОСМОПОР Е ПОВЯЗКА 15СМХ8СМ №25 П/ОПЕ...,Фармкомплект ООО,17.08.2021,13002787.0,16.08.2021,ФК003216604,0.040000,18.43,10%,1.84,20.28
2,3,201300053376,"ХАРТМАНН КОСМОПОР Е ПОВЯЗКА 7,2Х5СМ №50 П/ОПЕР...",Здравсервис,26.08.2021,13002938.0,26.08.2021,E12562599/37252019,0.040000,13.06,10%,1.31,14.36
3,4,201300019163,РИНГЕРА Р-Р 200МЛ. №24 Д/ИНФ. /КРАСФАРМА/,Катрен г.Химки,27.10.2020,13001682.0,27.10.2020,8856007-30,0.041667,30.12,10%,3.01,33.14
4,5,201300047442,МААЛОКС 15МЛ. №30 СУСП. Д/ПРИЕМА ВНУТРЬ ПАК.,Пульс,01.07.2021,13002158.0,30.06.2021,БРН00473831,0.066667,47.52,10%,4.75,52.27


Далее необходимо объединить данные с выгрузкой из СБИС.

In [169]:
doc_types = ["СчФктр", "УпдДоп", "УпдСчфДоп", "ЭДОНакл"]
# Столбцы для итогового датафрейма
final_cols = ['№ п/п', 'Штрих-код партии', 'Наименование товара', 'Поставщик',
'Дата приходного документа', 'Номер приходного документа',
'Дата накладной', 'Номер накладной', 'Номер счет-фактуры',
'Сумма счет-фактуры', 'Кол-во',
'Сумма в закупочных ценах без НДС', 'Ставка НДС поставщика',
'Сумма НДС', 'Сумма в закупочных ценах с НДС', 'Дата счет-фактуры', 'Сравнение дат']

In [206]:
merged_df = pd.merge(drugstores_df, sbis_df, left_on = "Номер накладной", right_on = 'Номер')\
              .rename(columns={'Номер' : 'Номер счет-фактуры', 'Сумма' : 'Сумма счет-фактуры', 'Дата' : 'Дата счет-фактуры'})
merged_df = merged_df[merged_df.Тип_документа.isin(doc_types)]
merged_df['Дата счет-фактуры'] = pd.to_datetime(merged_df['Дата счет-фактуры']).dt.strftime('%d.%m.%Y')
merged_df['Дата накладной'] = pd.to_datetime(merged_df['Дата накладной'], dayfirst=True).dt.strftime('%d.%m.%Y')
merged_df['Сравнение дат'] = (merged_df['Дата счет-фактуры'] == merged_df['Дата накладной']).map({True : '', False: 'Не совпадает!'})
merged_df = merged_df[final_cols]
merged_df.head()

C:\Users\фвьшт\AppData\Local\Temp\ipykernel_6252\2293421606.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_df['Дата счет-фактуры'] = pd.to_datetime(merged_df['Дата счет-фактуры']).dt.strftime('%d.%m.%Y')


,№ п/п,Штрих-код партии,Наименование товара,Поставщик,Дата приходного документа,Номер приходного документа,Дата накладной,Номер накладной,Номер счет-фактуры,Сумма счет-фактуры,Кол-во,Сумма в закупочных ценах без НДС,Ставка НДС поставщика,Сумма НДС,Сумма в закупочных ценах с НДС,Дата счет-фактуры,Сравнение дат
0,1,201300039533,СФМ ПЕРЧАТКИ СМОТР. ГОЛУБЫЕ Н/СТЕР. НИТРИЛ Р.L...,Катрен г.Химки,19.04.2021,13001265.0,18.04.2021,3541290-30,3541290-30,"31 259,43",0.020000,36.48,10%,3.65,40.13,18.04.2021,
2,1,201300039533,СФМ ПЕРЧАТКИ СМОТР. ГОЛУБЫЕ Н/СТЕР. НИТРИЛ Р.L...,Катрен г.Химки,19.04.2021,13001265.0,18.04.2021,3541290-30,3541290-30,"31 259,43",0.020000,36.48,10%,3.65,40.13,18.04.2021,
4,2,201300052321,ХАРТМАНН КОСМОПОР Е ПОВЯЗКА 15СМХ8СМ №25 П/ОПЕ...,Фармкомплект ООО,17.08.2021,13002787.0,16.08.2021,ФК003216604,ФК003216604,"7 618,89",0.040000,18.43,10%,1.84,20.28,17.08.2021,Не совпадает!
7,3,201300053376,"ХАРТМАНН КОСМОПОР Е ПОВЯЗКА 7,2Х5СМ №50 П/ОПЕР...",Здравсервис,26.08.2021,13002938.0,26.08.2021,E12562599/37252019,E12562599/37252019,"2 593,85",0.040000,13.06,10%,1.31,14.36,26.08.2021,
9,4,201300019163,РИНГЕРА Р-Р 200МЛ. №24 Д/ИНФ. /КРАСФАРМА/,Катрен г.Химки,27.10.2020,13001682.0,27.10.2020,8856007-30,8856007-30,"40 982,27",0.041667,30.12,10%,3.01,33.14,27.10.2020,


In [ ]:
def save_file(data):
    """
    Функция для сохранения файла c выгрузкой в папке Результат/{текущая_дата} с автоматическим созданием папок при их отсутствии.

    data: Данные, которые нужно сохранить
    """
    today_date = datetime.now().strftime('%Y-%m-%d')
    result_folder = os.path.join("Результат", today_date)
    os.makedirs(result_folder, exist_ok=True)  # Создание папок, если они не существуют
    output_file_path = os.path.join(result_folder, "Выгрузка - результат.xlsx")
    data.to_excel(output_file_path, index=False)
    print(f"Файл успешно сохранён по пути: {output_file_path}")

In [ ]:
save_file()

'Результат\\2025-03-30'